# Generación de archivos parquet VITALES Nacimientos

## Carga de los datos

Este jupyter notebook esta generando con todos los datos de Vitales Natalidad (2014 - 2023)

In [14]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasVITALES_Natalidad").getOrCreate()

In [13]:
#spark.stop()

In [15]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/vitales/vitales_parquet_*/")

df=df1_parquet

In [16]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 5458140
Columnas del DataFrame unido: ['PersonaID', 'FechaNacimientoAAAAMM', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaCertificadoAAAAMM', 'IpsDondeRealizaParto', 'Peso', '__null_dask_index__']


In [17]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 5458140


In [ ]:
Se verifica y el numero de registros corresponde exactaente al definido en el documento de exploración

In [18]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaID: string (nullable = true)
 |-- FechaNacimientoAAAAMM: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- RegimenAfiliacionSGSSS: string (nullable = true)
 |-- Administradora: string (nullable = true)
 |-- CodigoPertenenciaEtnica: string (nullable = true)
 |-- municipioResidencia: string (nullable = true)
 |-- SemanasGestacion: string (nullable = true)
 |-- EdadMadreMomentoDiagnostico: string (nullable = true)
 |-- FechaCertificadoAAAAMM: string (nullable = true)
 |-- IpsDondeRealizaParto: string (nullable = true)
 |-- Peso: string (nullable = true)
 |-- __null_dask_index__: long (nullable = true)

['PersonaID', 'FechaNacimientoAAAAMM', 'Sexo', 'RegimenAfiliacionSGSSS', 'Administradora', 'CodigoPertenenciaEtnica', 'municipioResidencia', 'SemanasGestacion', 'EdadMadreMomentoDiagnostico', 'FechaCertificadoAAAAMM', 'IpsDondeRealizaParto', 'Peso', '__null_dask_index__']


# Empieza el proceso de generación de los datos agrupados por atenciones para natalidades

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaID','FechaNacimientoAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaID').agg(
    F.first('FechaNacimientoAAAAMM').alias('FechaNacimientoAAAAMM'),
    F.first('RegimenAfiliacionSGSSS').alias('RegimenAfiliacionSGSSS'),
    F.first('Sexo').alias('Sexo'),
     F.first('EdadMadreMomentoDiagnostico').alias('EdadMadreMomentoDiagnostico'),
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100002222|               201503|        S - SUBSIDIADO|MASCULINO|                         21|              1|
|100007843|               201504|         E - EXCEPCION|MASCULINO|                         21|              1|
|100009759|               201411|        S - SUBSIDIADO|MASCULINO|                         20|              1|
|100009965|               202112|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100010646|               201910|      C - CONTRIBUTIVO|MASCULINO|                         25|              1|
| 10001567|               202101|      C - CONTRIBUTIVO| FEMENINO|                         35|              1|
|

+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100002222|               201503|        S - SUBSIDIADO|MASCULINO|                         21|              1|
|100007843|               201504|         E - EXCEPCION|MASCULINO|                         21|              1|
|100009759|               201411|        S - SUBSIDIADO|MASCULINO|                         20|              1|
|100009965|               202112|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100010646|               201910|      C - CONTRIBUTIVO|MASCULINO|                         25|              1|
| 10001567|               202101|      C - CONTRIBUTIVO| FEMENINO|                         35|              1|
|

In [21]:
# Contar el número de registros
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 4510549


In [22]:
 df_final =grouped_df


In [23]:
# Contar el número de registros 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 4510549


In [24]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+---------------------+----------------------+---------+---------------------------+---------------+
|PersonaID|FechaNacimientoAAAAMM|RegimenAfiliacionSGSSS|     Sexo|EdadMadreMomentoDiagnostico|personaid_count|
+---------+---------------------+----------------------+---------+---------------------------+---------------+
|100002222|               201503|        S - SUBSIDIADO|MASCULINO|                         21|              1|
|100007843|               201504|         E - EXCEPCION|MASCULINO|                         21|              1|
|100009759|               201411|        S - SUBSIDIADO|MASCULINO|                         20|              1|
|100009965|               202112|      C - CONTRIBUTIVO| FEMENINO|                         27|              1|
|100010646|               201910|      C - CONTRIBUTIVO|MASCULINO|                         25|              1|
| 10001567|               202101|      C - CONTRIBUTIVO| FEMENINO|                         35|              1|
|

# Empieza el proceso de generación de tablas VITALES Nacimientos

# Pagina 1 - Power BI

In [25]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

## 1. Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS

In [26]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
#df2.select('FechaNacimientoAAAAMM').show(100, truncate=False)

In [27]:
from pyspark.sql import functions as F

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaID', 'FechaNacimientoAAAAMM', 'RegimenAfiliacionSGSSS', 'Sexo','EdadMadreMomentoDiagnostico', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaNacimientoAAAAMM' a año y mes correctamente
df_filtered = df_filtered.withColumn(
    'Año', F.substring(F.col('FechaNacimientoAAAAMM'), 1, 4).cast('int')
).withColumn(
    'Mes', F.substring(F.col('FechaNacimientoAAAAMM'), 5, 2).cast('int')
)

# Filtrar valores atípicos y nulos
df_filtered = df_filtered.filter(
    (F.col('Año').isNotNull()) & (F.col('Mes').isNotNull()) & (F.col('Año') > 1900)
)

# Agrupar por Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS
result_df = df_filtered.groupBy('Año', 'Mes', 'Sexo','EdadMadreMomentoDiagnostico', 'RegimenAfiliacionSGSSS').agg(
    F.countDistinct('PersonaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------+---------------------------+----------------------+------------------+--------------------+
| Año|Mes|     Sexo|EdadMadreMomentoDiagnostico|RegimenAfiliacionSGSSS|Número_de_Personas|Número_de_Atenciones|
+----+---+---------+---------------------------+----------------------+------------------+--------------------+
|2016| 10| FEMENINO|                         36|        S - SUBSIDIADO|               200|                 228|
|2022|  8|MASCULINO|                         26|        S - SUBSIDIADO|               514|                 725|
|2018|  7|MASCULINO|                         21|        S - SUBSIDIADO|               782|                1057|
|2017|  6| FEMENINO|                         19|     NR - NO REPORTADO|                27|                  32|
|2019|  2| FEMENINO|                         39|      C - CONTRIBUTIVO|               119|                 129|
|2017| 11| FEMENINO|                         33|      C - CONTRIBUTIVO|               364|              

In [28]:
#se escribe el parquet tabla 1- Año, Mes, Sexo, EdadMadreMomentoDiagnostico, y RegimenAfiliacionSGSSS
result_df.write.parquet("hdfs:/analyticsdata/1.VITALES_Nacimientos_Tabla_anio_mes_Sexo_Edad_RegimenAfiliacionSGSSS_Todos.parquet") 

In [30]:
spark.stop()